In [55]:
!pip install google-auth google-auth-httplib2 google-api-python-client google-auth-oauthlib transformers

  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata (2.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 275.9 kB/s eta 0:00:00a 0:00:01
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.

In [77]:
!pip install psycopg2-binary
!pip install python-dotenv
!pip install tqdm 
!pip install pyannote.audio


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached pyannote.audio-3.3.1-py2.py3-none-any.whl.metadata (11 kB)
  Using cached asteroid_filterbanks-0.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
  Using cached lightning-2.3.1-py3-none-any.whl.metadata (54 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached pyannote.core-5.0.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached pyannote.database-5.1.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached pyannote.metrics-3.2.1-py3-none-any.whl.metadata (1.3 kB)
  Using cached pyannote.pipeline-3.0.1-py3-none-any.whl.metadata (897 bytes)
  Using cached pytorch_metric

In [78]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import collect_segments
from util import split_audio_files

ModuleNotFoundError: No module named 'pyannote'

In [ ]:
from pydub import AudioSegment
import pandas as pd

# Function to get audio duration from an audio file
def get_audio_duration(audio_filename):
    try:
        audio = AudioSegment.from_file(audio_filename)
        duration_in_sec = len(audio) // 1000  # Duration in seconds
        duration = pd.to_datetime(duration_in_sec, unit='s').strftime('%H:%M:%S')  # Duration in H:M:S format
        return duration
    except Exception as e:
        print(f"Error fetching duration for {audio_filename}: {e}")
        return None

In [ ]:
import subprocess

def download_s3_file(s3_path, local_filename):
    """
    Downloads a file from S3 using AWS CLI.

    Args:
    - s3_path (str): S3 path of the file to download.
    - local_filename (str): Local filename to save the downloaded file.

    Returns:
    - bool: True if download successful, False otherwise.
    """
    try:
        s3_bucket = "monlam.ai.stt"  # Replace with your S3 bucket name

        # Run aws s3 cp command to download the file
        download_command = f"aws s3 cp s3://{s3_bucket}/{s3_path} {local_filename}"
        subprocess.run(download_command, shell=True, check=True)
        print(f"Downloaded {s3_path} to {local_filename}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error downloading {s3_path}: {e}")
        return False

# Example usage:
from_id = 305
to_id = 319

durations = []

df = read_spreadsheet(sheet_id="1aShlQ9I2FS_PX_8Ukvc1nr8qOiMePdAzO-hs6-vWKko")

for index, row in df.iterrows():
    if not isinstance(row['File Name'], str) or not isinstance(row['STT_NS_P0000'], str):
        break
    id = row['STT_NS_P0000']
    s3_path = row['File Name']
    sr_no = row.iloc[0]
    if sr_no >= from_id and sr_no <= to_id:
        local_filename = f"audio_from_s3/{id}.mp3"
        download_s3_file(s3_path, local_filename)
        audio_filename = f"full_audio/{id}"
        duration = get_audio_duration(audio_filename)
        print(f"Audio duration for {audio_filename}: {duration}")
        durations.append((id, duration))



In [ ]:
# Convert list to DataFrame and save to CSV
df_durations = pd.DataFrame(durations, columns=['File Name', 'Duration'])
df_durations.to_csv('nw_gh_audio_durations.csv', index=False)

print("Durations have been saved to audio_durations.csv")